#Telecom Domain Read & Write Ops Assignment - Building Datalake & Lakehouse
This notebook contains assignments to practice Spark read options and Databricks volumes. <br>
Sections: Sample data creation, Catalog & Volume creation, Copying data into Volumes, Path glob/recursive reads, toDF() column renaming variants, inferSchema/header/separator experiments, and exercises.<br>

![](https://fplogoimages.withfloats.com/actual/68009c3a43430aff8a30419d.png)
![](https://theciotimes.com/wp-content/uploads/2021/03/TELECOM1.jpg)

##First Import all required libraries & Create spark session object

##1. Write SQL statements to create:
1. A catalog named telecom_catalog_assign
2. A schema landing_zone
3. A volume landing_vol
4. Using dbutils.fs.mkdirs, create folders:<br>
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/
/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/
5. Explain the difference between (Just google and understand why we are going for volume concept for prod ready systems):<br>
a. Volume vs DBFS/FileStore<br>
b. Why production teams prefer Volumes for regulated data<br>

In [0]:
%sql
--1. Write SQL statements to create:
--1.1. A catalog named telecom_catalog_assign
CREATE CATALOG IF NOT EXISTS telecom_catalog_assign;

--1.2. A schema landing_zone
CREATE SCHEMA IF NOT EXISTS telecom_catalog_assign.landing_zone;

--1.3. A volume landing_vol
CREATE VOLUME IF NOT EXISTS telecom_catalog_assign.landing_zone.landing_vol;


In [0]:
%sql
--drop catalag even data is present inside - for reference
--DROP CATALOG IF EXISTS telecom_catalog_assign CASCADE;

In [0]:
# 1.4. Using dbutils.fs.mkdirs, create folders:
# /Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/ 
# /Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/ 
# /Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/

dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage")
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1")

In [0]:
# # We can create multiple folders in one go by using a loop, instead of writing dbutils.fs.mkdirs() one by one.
# folders = ['/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1 ','/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2']

# for folder in folders:
#   dbutils.fs.mkdirs(folder)

# Explain the difference between dbfs and volume in databricks:
##### 1. DBFS (Databricks File System)<br>
- A legacy abstraction layer over cloud object storage (ADLS, S3, GCS).
- Makes cloud storage look like a local file system.
- Tied to the workspace, not Unity Catalog
- Limited governance and security
- Often accessed using dbutils.fs
- Common in older Databricks projects

##### 2. Volumes (Unity Catalog Volumes)
- A modern, governed storage abstraction introduced with Unity Catalog. 
- Provides secure, managed access to cloud storage.
- Governed by Unity Catalog. 
- Fine-grained permissions (READ, WRITE). 
- Works seamlessly with SQL, Python, Spark

##Data files to use in this usecase:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''

tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
'''

In [0]:
customer_csv = '''
101,Arun,31,Chennai,PREPAID
102,Meera,45,Bangalore,POSTPAID
103,Irfan,29,Hyderabad,PREPAID
104,Raj,52,Mumbai,POSTPAID
105,,27,Delhi,PREPAID
106,Sneha,abc,Pune,PREPAID
'''
usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count
101\t320\t1500\t20
102\t120\t4000\t5
103\t540\t600\t52
104\t45\t200\t2
105\t0\t0\t0
'''
tower_logs_region1 = '''event_id|customer_id|tower_id|signal_strength|timestamp
5001|101|TWR01|-80|2025-01-10 10:21:54
5004|104|TWR05|-75|2025-01-10 11:01:12
5002|104|TWR06|-75|2025-01-10 11:01:12
5003|104|TWR02|-80|2025-01-10 11:01:12
5005|104|TWR03|-75|2025-01-10 11:01:12
5006|104|TWR04|-75|2025-01-10 11:01:12
'''

##2. Filesystem operations
1. Write code to copy the above datasets into your created Volume folders:
Customer → /Volumes/.../customer/
Usage → /Volumes/.../usage/
Tower (region-based) → /Volumes/.../tower/region1/ and /Volumes/.../tower/region2/

2. Write a command to validate whether files were successfully copied

In [0]:
# 2.1. Write code to copy the above datasets into your created Volume folders:
# dbutils.fs.put(path: str, contents: str[file content or variable], overwrite: bool)
dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv', customer_csv, True)

dbutils.fs.put('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv', usage_tsv, True)

dbutils.fs.put(f"/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/ericsson/tower_region1_ericsson.csv", tower_logs_region1,overwrite = True)


In [0]:
# 2.2. Write a command to validate whether files were successfully copied

print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/'))
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/'))
print(dbutils.fs.ls('/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/'))

##3. Directory Read Use Cases
1. Read all tower logs using:
Path glob filter (example: *.csv)
Multiple paths input
Recursive lookup

2. Demonstrate these 3 reads separately:
Using pathGlobFilter
Using list of paths in spark.read.csv([path1, path2])
Using .option("recursiveFileLookup","true")

3. Compare the outputs and understand when each should be used.

In [0]:
#3.1
Tower_log_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower", header=True, inferSchema=True, pathGlobFilter="*.csv", recursiveFileLookup=True, sep="|")
display(Tower_log_df)

# spark.read.format(csv).option("header", "true").option("inferSchema", "true").option("recursiveFileLookup","true").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower").display()

#3.2
tower_log_df1 = spark.read.csv(["/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/", "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower_all_data/"], header=True, inferSchema=True, recursiveFileLookup=True, pathGlobFilter="*.csv", sep="|")
display(tower_log_df1)

##4. Schema Inference, Header, and Separator
1. Try the Customer, Usage files with the option and options using read.csv and format function:<br>
header=false, inferSchema=false<br>
or<br>
header=true, inferSchema=true<br>
2. Write a note on What changed when we use header or inferSchema  with true/false?<br>
3. How schema inference handled “abc” in age?<br>

In [0]:
#4.1
customer_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer", header=True, inferSchema=True).printSchema()
display(customer_df)

#4.2
#When Header is false and inferschema is false data format is taken as string, 
#When Header is true and inferschema is true is data format is taken as data format given in data file
#when either header or inferschema is false, data format is taken as string

# 4.3. How schema inference handled “abc” in age?
# Age contains both integer and string value so it is considerign the age column as string[string + integer = string]


##5. Column Renaming Usecases
1. Apply column names using string using toDF function for customer data
2. Apply column names and datatype using the schema function for usage data
3. Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data 

In [0]:
#5.1
dbutils.fs.rm("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/", True)
#custid,name,age,city,plan
customer1_data = '''
1,"divya",20,"bangalore",postpaid
2,"anu",24,"chennai",postpaid
3,"sita",25,"hyderabad",postpaid
'''
dbutils.fs.mkdirs("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/")
dbutils.fs.put("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/customer1.csv", customer1_data, True)

customer_df1 = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/customer1.csv",header=False, inferSchema=True).toDF("customer_id","user_name","age","city","plan")
#if we user todf() function no need to use of header = true
# display(customer_df1)

customer_df1.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer1/customer_csv_folder")

In [0]:
#5.2
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType, TimestampType
usage_data_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("voice_mins", IntegerType(), False),
    StructField("data_mb", IntegerType(), False),
    StructField("sms_count", IntegerType(), False),
])

usage_df = spark.read.schema(usage_data_schema).options(header="True", sep="\t").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.csv")
#while using schema we need to use options() function

display(usage_df)

In [0]:
#5.3
usage_data_schema = StructType([
    StructField("event_id", IntegerType(), True),
    StructField("customer_id", LongType(), False),
    StructField("tower_id", StringType(), False),
    StructField("signal_strength", LongType(), False),
    StructField("timestamp", TimestampType(), False)
])

tower_df1 = spark.read.schema(usage_data_schema).options(header="True", sep="|").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1/tower_logs_region1.csv")
display(tower_df1)